In [ ]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import exoplanet as xo
import astropy.units as u
from astropy.io import fits
from astropy.timeseries import BoxLeastSquares
%matplotlib inline

In [ ]:
from lightkurve import KeplerTargetPixelFile
import lightkurve as lk
tpf = KeplerTargetPixelFile("file location") #enter file location and be sure that it is a TargetPixelFile
tpf

In [ ]:
tpf.plot()

In [ ]:
lcur = tpf.to_lightcurve()

In [ ]:
lcur.scatter();

In [ ]:
flat_lc = lcur.flatten(window_length=value) #value should be an odd integer
flat_lc.scatter();

In [ ]:
mask = (flat_lc.time < x) | (flat_lc.time > y) #x equals starting value of the mask and y equals ending value
masked_lc = flat_lc[mask]
masked_lc.scatter();

In [ ]:
clipped_lc = mask_lc.remove_outliers(sigma=value) #value in this line equals to the strenght of the clipping
clipped_lc.scatter()

In [ ]:
lc = clipped_lc

In [ ]:
def lc_analysis(x, y, yerr, periods=None):
    """Plots the timeseries for given periods"""
    if periods is None:
        print("""You must specify the periods limits to search for transits.
                 Periods format: [[min_period1, ...,min_periodn]]
                                  [max_period1, ...,max_periodn]]""")
        return
    # Plot relative flux time series
    print("\n Light curve flux time series:")
    plt.figure(figsize=(16,5))
    plt.plot(x, y, "k", linewidth=0.5)
    plt.xlabel("Time [BKJD]")
    plt.ylabel("Relative Flux")
    plt.show()
    # Now use a box least squares periodogram to find transits
    # Periods format: [[min_period1, ...,min_periodn]]
    #                  [max_period1, ...,max_periodn]]
    plt.figure(figsize=(16, 5))
    print("\n Harmonics are in highlighted in red.")
    
    for column in range(np.shape(periods)[1]):
        periodogram = xo.estimators.bls_estimator(x, y, yerr, min_period=periods[0][column], max_period=periods[1][column])
        peak = periodogram["peak_info"]
        t0_guess = peak["transit_time"]  # Time of first transit guesses
        period_guess = peak["period"]  # Period guesses
        depth_guess = peak["depth"]  # Depths guesses
        plt.plot(periodogram["bls"].period, periodogram["bls"].power, "k", lw=0.5)
        label = np.round(period_guess, decimals=3)
        plt.axvline(period_guess*4, alpha=0.3, linewidth=5, color="red")
        plt.axvline(period_guess*3, alpha=0.3, linewidth=5, color="red")
        plt.axvline(period_guess*2, alpha=0.3, linewidth=5, color="red")
        plt.axvline(period_guess, alpha=0.3, linewidth=5)
        plt.axvline(period_guess/2, alpha=0.3, linewidth=5, color="red")
        plt.axvline(period_guess/4, alpha=0.3, linewidth=5, color="red")
        plt.axvline(period_guess/8, alpha=0.3, linewidth=5, color="red")
        txt = f"Period = {label}[BKJD]"
        plt.annotate(txt, (0, 0), xycoords="axes fraction", xytext=(5, 250 - 15 * column),
                     textcoords="offset points", ha="left",va="bottom", fontsize=12)
    plt.xlabel("Period [BKJD]") #Time needs to be changes depending on the format. BJD, BTJD etc.
    plt.ylabel("BLS Power")
    plt.xlim(periods[0].min(), periods[1].max())
    plt.grid()
    plt.show()
    return x, y, yerr

In [ ]:
x_lc, y_lc, yerr_lc = lc_analysis(lc.time, lc.flux, lc.flux_err, periods=np.array([[value1], [value2]])) #change value 1 and 2 dependng on the period of the system

In [ ]:
clipped_lc.to_csv("example.csv") #export the calibrated light curve in csv format

In [ ]:
clipped_lc.to_fits("example.fits") #export the calibrated light curve in fits format